# Driving Scene Segmentation

This project demostrates the steps to run DeepLab semantic scene segmentation model on a sample image from MIT Driving Scene Segmentation Dataset.

In [1]:
import tensorflow as tf
import os
import tarfile
import numpy as np
from PIL import Image
import cv2 as cv
import warnings
import dlm
warnings.simplefilter("ignore", DeprecationWarning)

### Build the model

**[DeepLab](https://github.com/tensorflow/models/tree/master/research/deeplab)** is a state-of-art deep learning model for semantic image segmentation, where the goal is to assign semantic labels (e.g., person, dog, cat and so on) to every pixel in the input image. Some segmentation results on Flickr images:

<p align="center">
    <img src="https://github.com/tensorflow/models/blob/master/research/deeplab/g3doc/img/vis1.png?raw=true" width=600></br>
    <img src="https://github.com/tensorflow/models/blob/master/research/deeplab/g3doc/img/vis2.png?raw=true" width=600></br>
</p>

In the driving context, we aim to obtain a semantic understanding of the front driving scene throught the camera input. This is important for driving safety and an essential requirement for all levels of autonomous driving. The first step is to build the model and load the pre-trained weights. In this demo, we use the model checkpoint trained on [Cityscapes](https://www.cityscapes-dataset.com/) dataset.

<p align="center">
    <img src="https://www.cityscapes-dataset.com/wordpress/wp-content/uploads/2015/07/muenster00.png" width=600></br>
    <img src="https://www.cityscapes-dataset.com/wordpress/wp-content/uploads/2015/07/zuerich00.png" width=600></br>
</p>


In [2]:
class DeepLabModel(object):
    """Class to load deeplab model and run inference."""

    FROZEN_GRAPH_NAME = 'frozen_inference_graph'

    def __init__(self, tarball_path):
        """Creates and loads pretrained deeplab model."""
        self.graph = tf.Graph()
        graph_def = None
        # Extract frozen graph from tar archive.
        tar_file = tarfile.open(tarball_path)
        for tar_info in tar_file.getmembers():
            if self.FROZEN_GRAPH_NAME in os.path.basename(tar_info.name):
                file_handle = tar_file.extractfile(tar_info)
                graph_def = tf.compat.v1.GraphDef.FromString(file_handle.read())
                break
        tar_file.close()

        with self.graph.as_default():
            tf.import_graph_def(graph_def, name='')
        self.sess = tf.compat.v1.Session(graph=self.graph)

    def run(self, image, INPUT_TENSOR_NAME = 'ImageTensor:0', OUTPUT_TENSOR_NAME = 'SemanticPredictions:0'):
        target_size = (2049,1025)  # size of Cityscapes images
        resized_image = image.convert('RGB').resize(target_size, Image.ANTIALIAS)
        batch_seg_map = self.sess.run(
            OUTPUT_TENSOR_NAME,
            feed_dict={INPUT_TENSOR_NAME: [np.asarray(resized_image)]})
        seg_map = batch_seg_map[0]  # expected batch size = 1
        if len(seg_map.shape) == 2:
            seg_map = np.expand_dims(seg_map,-1)  # need an extra dimension for cv.resize
        seg_map = cv.resize(seg_map, (image.size), interpolation=cv.INTER_NEAREST)
        return seg_map

### Load the model from a frozen graph

In [3]:
download_path = r'C:\Users\sakth\PycharmProjects\python-projects\palan_project\projects\INTERNSHIP PROJECTS\environment detection and segmentation\deeplab_model.tar.gz'

MODEL = DeepLabModel(download_path)
print('model loaded successfully!')

model loaded successfully!


### Run on the sample image
The sample image is frame #0 in the MIT Driving Scene Segmentation (DriveSeg) Dataset.

In [4]:
SAMPLE_IMAGE = r'C:\Users\sakth\PycharmProjects\python-projects\palan_project\projects\INTERNSHIP PROJECTS\environment detection and segmentation\environment_detection_and_segmentation.png'
print('running deeplab on the sample image...')
#dlm.run_visualization(SAMPLE_IMAGE)

running deeplab on the sample image...


In [5]:
TEST_IMAGE = r'C:\Users\sakth\PycharmProjects\python-projects\palan_project\projects\INTERNSHIP PROJECTS\environment detection and segmentation\test_local.jpg'
print('running deeplab on the sample image...')
#dlm.run_visualization(TEST_IMAGE)

running deeplab on the sample image...
